Based on the Prototype 2 (AD-PMM) algorithm of Shefi and Teboulle 2014, for convex QPs of the form:
$$
\begin{align}
\text{mimimise } &\frac{1}{2} x^T P x + c^T x \\
\text{subject to } & Ax + s = b \\
& s \in \mathcal{K}
\end{align}
$$
where $P \in \mathbb{S}^{n}_+$, $c \in \mathbb{R}^n$, $A \in \mathbb{R}^{m \times n}$, and $b \in \mathbb{R}^m$.

For now I only have code for $\mathcal{K} = \mathbb{R}^n_+$. Julia code for projections onto many cones of interest is available in the COSMO stuff.


For a generic square symmetric matrix $P$, we use the notation/partition
$$
P = D(P) + R(P),
$$
where $D(P)$ is diagonal with the same entries as those in the main diagonal of $P$. $R(P)$ is the remainder, entries not in the diagonal of $P$.

In [65]:
# Get the required libraries
using LinearAlgebra, Printf
using JuMP
using Clarabel, ClarabelBenchmarks

ArgumentError: ArgumentError: Package ClarabelBenchmarks not found in current path.
- Run `import Pkg; Pkg.add("ClarabelBenchmarks")` to install the ClarabelBenchmarks package.

In [2]:
# Define the QP problem data
struct QPProblem
    P::Matrix{Float64}  # PSD matrix in R^{n x n}
    c::Vector{Float64}  # Vector in R^n
    A::AbstractMatrix{Float64}  # Matrix in R^{m x n}
    b::Vector{Float64}  # Vector in R^m
    M1::Matrix{Float64}
    τ::Float64
    ρ::Float64
    
    
    # Constructor with a check for P being PSD
    function QPProblem(P::Matrix{Float64}, c::Vector{Float64}, A::Matrix{Float64}, b::Vector{Float64}, M1::Matrix{Float64}, τ::Float64, ρ::Float64)
        # Check if P is symmetric and positive semidefinite
        if !issymmetric(P)
            error("Matrix P must be symmetric.")
        end
        
        # Check positive semidefiniteness of P by verifying all eigenvalues are non-negative
        if minimum(eigvals(P)) < 0
            error("Matrix P must be positive semidefinite (PSD).")
        end
        
        # Check for M1 being PSD
        n = size(A)[2]
        if minimum(eigvals(M1)) < 0
            error("Matrix M1 must be PSD. Current smallest eigval: $minimum(eigvals(M1))")
        end
        
        # Create the instance of the struct
        new(P, c, A, b, M1, τ, ρ) # what to store in struct, to be accessed later
    end
end

In [7]:
# Auxiliary function, given a square matrix A, it returns a new matrix
# whose diagonal is equal to the diagonal of A, and all other entries are zero
function diag_part(A::Matrix{Float64})
    return Diagonal(diag(A))
end

# Auxiliary function, given a square matrix A, it returns a new matrix
# whose diagonal is zero and where all other entries are equal to those of A
function off_diag_part(A::Matrix{Float64})
    return A - diag_part(A)
end;

In [57]:
# Initialise QP data
n = 3  # number of variables
m = 3  # number of (inequality) constraints

P = [5.0 0 1; 0 6 6; 1 6 8]
c = [1.0, 2, 3]
b = -1 * ones(Float64, m)
A = Matrix{Float64}(I, n, n)

# Step size parameters chosen by the user
ρ = 1.0 # (dual)

# Choose algorithm based on M1 construction
variant = 1 # int in {1, 2, 3, 4}

if variant == 1
    take_away = off_diag_part(P + ρ * A' * A)
elseif variant == 2
    take_away = P + ρ * A' * A
elseif variant == 3
    take_away = P + off_diag_part(ρ * A' * A)
elseif variant == 4
    take_away = off_diag_part(P) + ρ * A' * A
else
    error("Invalid variant.")
end

# Choose primal step size as 90% of maximum allowable while to keep M1 PSD
τ = 1 / maximum(eigvals(take_away))
M1 = (1 / τ) * Matrix{Float64}(I, n, n) - take_away
τ

0.16439898730535732

# Solve with our method

In [58]:
#TODO: implement residual calculation
# termination criteria; look at COSMO paper.

# Can later try against SCS or COSMO

# mul! function for multiplication? or better to just count iterations for now (code is more readable)
# benchmarks from Clarabel paper --- need to extract problem data, maybe tricky

# Create a problem instance
problem = QPProblem(P, c, A, b, M1, τ, ρ)

# Initialize sequences x, s, and y
x = ones(n)
s = ones(m)
y = ones(m)

# Define the iteration function
function iterate!(problem::QPProblem, x::Vector{Float64}, s::Vector{Float64}, y::Vector{Float64})
    # Unpack problem data for easier reference
    P, c, A, b, τ, ρ, M1 = problem.P, problem.c, problem.A, problem.b, problem.τ, problem.ρ, problem.M1
    
    ### Step 1: Update x ###
    # Kept in general form here; may not be a diagonal system, depending on choice of M1
    x_new = x - (M1 + P + ρ * A' * A) \ (P * x + c + A' * (y + ρ * (A * x + s - b)))

    ### Step 2: Update s ###
    # Project basic update onto nonnegative orthant
    s_new = max.(b - A * x_new - y / ρ, 0)

    ### Step 3: Update y ###
    y_new = y + ρ * (A * x_new + s_new - b)

    # Update the sequences
    x .= x_new
    s .= s_new
    y .= y_new
end

# Main iteration loop
max_iters = 200  # Maximum number of iterations (adjust as needed)
for k in 0:max_iters
    # Print each iteration result with formatted output on a single line
    if k % 20 == 0
        @printf("Iteration %4.1d: x = [%s]  s = [%s]  y = [%s]\n",
            k,
            join(map(xi -> @sprintf("%12.5e", xi), x), ", "),
            join(map(zi -> @sprintf("%12.5e", zi), s), ", "),
            join(map(yi -> @sprintf("%12.5e", yi), y), ", "))
    end
    
    iterate!(problem, x, s, y)
end

# Print final solution on a single line
println("\nPrototype FOM results:")
@printf("x = [%s]  s = [%s]  y = [%s]\n",
    join(map(xi -> @sprintf("%12.5e", xi), x), ", "),
    join(map(zi -> @sprintf("%12.5e", zi), s), ", "),
    join(map(yi -> @sprintf("%12.5e", yi), y), ", "))

Iteration    0: x = [ 1.00000e+00,  1.00000e+00,  1.00000e+00]  s = [ 1.00000e+00,  1.00000e+00,  1.00000e+00]  y = [ 1.00000e+00,  1.00000e+00,  1.00000e+00]
Iteration   20: x = [-9.68611e-01, -7.98459e-01, -8.34190e-01]  s = [ 0.00000e+00,  0.00000e+00,  0.00000e+00]  y = [ 4.65015e+00,  7.65513e+00,  9.57619e+00]
Iteration   40: x = [-9.94746e-01, -9.64497e-01, -9.51961e-01]  s = [ 0.00000e+00,  0.00000e+00,  0.00000e+00]  y = [ 4.91662e+00,  9.45131e+00,  1.14457e+01]
Iteration   60: x = [-9.99277e-01, -9.94162e-01, -9.86897e-01]  s = [ 0.00000e+00,  0.00000e+00,  0.00000e+00]  y = [ 4.98193e+00,  9.87972e+00,  1.18662e+01]
Iteration   80: x = [-9.99855e-01, -9.98715e-01, -9.96944e-01]  s = [ 0.00000e+00,  0.00000e+00,  0.00000e+00]  y = [ 4.99631e+00,  9.97465e+00,  1.19670e+01]
Iteration  100: x = [-9.99943e-01, -9.99581e-01, -9.99406e-01]  s = [ 0.00000e+00,  0.00000e+00,  0.00000e+00]  y = [ 4.99923e+00,  9.99451e+00,  1.19921e+01]
Iteration  120: x = [-9.99979e-01, -9.99862e-0

# Solve with OSQP

In [29]:
using OSQP
using SparseArrays

# Convert problem data to sparse type
P_osqp = sparse(P)
A_osqp = sparse(A)
# Create OSQP object
prob = OSQP.Model()
# Set up workspace and change alpha parameter
OSQP.setup!(prob; P=P_osqp, q=c, A=A_osqp, u=b, alpha=1, verbose=false)
# Solve problem
results = OSQP.solve!(prob);

# Print OSQP results
println("OSQP results:")
println("x = [", join(map(v -> @sprintf("%12.5e", v), results.x), ", "), "]")
println("y = [", join(map(v -> @sprintf("%12.5e", v), results.y), ", "), "]")

OSQP results:
x = [-1.00000e+00, -1.00000e+00, -1.00000e+00]
y = [ 5.00000e+00,  1.00000e+01,  1.20000e+01]
